In [30]:
import keras
import math
import pandas as pd
import numpy as np
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error as mse
import time
import pickle
pd.set_option('display.max_rows', 500)
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

with open('resultsT123.pickle', 'rb') as f:
    results = pickle.load(f)
 
del data['T3'] #unusable trajectory, not enough intersections.

KeyError: 'T3'

In [2]:
#used functions:

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

def init_model(train_X):
    model = keras.models.Sequential()
    model.add(keras.layers.LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(keras.layers.Dense(1))
    model.compile(loss=keras.losses.MeanSquaredLogarithmicError(), optimizer='adam')
    return model

def train_model(train_X,train_y,test_X,test_y,model):
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
    history = model.fit(train_X, train_y, epochs=250, batch_size=64, validation_data=(test_X, test_y), 
                        verbose=0,callbacks=[es], shuffle=False)

    return history, model


In [4]:
target_intersections={"T1":{"North":"K504", "South":"K198"},
                      "T2":{"North":"K703", "South":"K206"}}
thresholds = [1,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0]
def build_LSTMs(results,targets=target_intersections,thesholds=thresholds):
    """Takes the entire results dict and builds LSTM models for each trajectory and direction"""
    errors={}
    dfs={}
    intersection_arrays = []
    for trajectory in results.keys():
        errors[trajectory]={}
        print("Starting trajectory: {}".format(trajectory))
        for direction in results[trajectory]:
            errors[trajectory][direction]={}
            print("Starting direction: {}".format(direction))
            lof = results[trajectory][direction]['lof_df'] # load LOF DF for cors
            data = pd.DataFrame()
            for intersection_name in results[trajectory][direction]['raw']:
                intersection = results[trajectory][direction]['raw'][intersection_name]
                intersection = intersection.rename(columns={"cars": intersection_name})
                intersection = intersection.set_index(pd.DatetimeIndex(intersection['timestamp']))
                intersection = intersection.drop(columns=['timestamp'])
                data = pd.merge(data, intersection, left_index=True, right_index=True, how='outer')
            data.dropna(inplace=True)
            dfs["{}_{}".format(trajectory,direction)] = data

            for threshold in thresholds:
                errors[trajectory][direction][threshold] ={}
                print("Threshold: {}".format(threshold))
                #grab relevant intersections:
                target = target_intersections[trajectory][direction]

                isct_inc = lof.corr()[lof.corr()[target]>=threshold].index.tolist()
                if (isct_inc in intersection_arrays) & (threshold != 0.4):
                    print("same intersections, copy MSE")
                    errors[trajectory][direction][threshold]['MSE'] = mse(preds,val_y)
                else:    
                    print("Intersections included: {}".format(len(isct_inc)))
                    errors[trajectory][direction][threshold]['intersections'] = isct_inc

                    df = data[isct_inc].copy(deep=True)

                    #move target var to front of DF
                    df = df[ [target] + [ col for col in df.columns if col != target ] ]

                    #do scaling:
                    scaler = StandardScaler()
                    df= scaler.fit_transform(df.values)
                    df_test = df[math.ceil(len(df)*0.8):]
                    df = df[:math.ceil(len(df)*0.8)]

                    # specify the lag sequence
                    sequence_length = 12
                    n_features = len(isct_inc)
                    # frame as supervised learning
                    reframed = series_to_supervised(df, sequence_length, 1)
                    # split into train and test sets
                    values = reframed.values
                    n_train_hours = math.ceil(len(df)*0.8)
                    train = values[:n_train_hours, :]
                    test = values[n_train_hours:, :]
                    # split into input and outputs
                    n_obs = sequence_length * n_features
                    train_X, train_y = train[:, :n_obs], train[:, -n_features]
                    test_X, test_y = test[:, :n_obs], test[:, -n_features]
                    # reshape input to be 3D [samples, timesteps, features]
                    train_X = train_X.reshape((train_X.shape[0], sequence_length, n_features))
                    test_X = test_X.reshape((test_X.shape[0], sequence_length, n_features))
                    #init model & train stuff
                    model=init_model(train_X)
                    history,model = train_model(train_X,train_y,test_X,test_y,model)

                    #eval:
                    reframed = series_to_supervised(df_test, sequence_length, 1)
                    values = reframed.values
                    train = values
                    n_obs = sequence_length * n_features
                    val_X, val_y = train[:, :n_obs], train[:, -n_features]
                    val_X = val_X.reshape((val_X.shape[0], sequence_length, n_features))
                    preds = model.predict(val_X)
                    preds =preds.reshape(len(df_test)-sequence_length)

                    model=init_model(train_X)
                    history,model = train_model(train_X,train_y,test_X,test_y,model)

                    #eval:
                    reframed = series_to_supervised(df_test, sequence_length, 1)
                    values = reframed.values
                    train = values
                    n_obs = sequence_length * n_features
                    val_X, val_y = train[:, :n_obs], train[:, -n_features]
                    val_X = val_X.reshape((val_X.shape[0], sequence_length, n_features))
                    preds = model.predict(val_X)
                    preds =preds.reshape(len(df_test)-sequence_length)


                    errors[trajectory][direction][threshold]['df'] = pd.DataFrame({"Real":val_y,"Predicted":preds,'SE':(val_y-preds)**2})
                    errors[trajectory][direction][threshold]['MSE'] = mse(preds,val_y)
                #save model
                
                path= '.\\{}\\{}\\{}\\'.format(trajectory,direction,threshold)
                if not os.path.exists(path):
                    os.makedirs(path)
                model.save(path+"model")
                
                print(mse(preds,val_y))
                intersection_arrays.append(isct_inc)

    return errors,dfs #errors
lstm, data = build_LSTMs(data)


Starting trajectory: T1
Starting direction: North
Threshold: 1
Intersections included: 1
Epoch 00049: early stopping
Epoch 00058: early stopping


INFO:tensorflow:Assets written to: .\T1\North\1\model\assets


INFO:tensorflow:Assets written to: .\T1\North\1\model\assets


0.3427815773198963
Threshold: 0.8
same intersections, copy MSE


INFO:tensorflow:Assets written to: .\T1\North\0.8\model\assets


...

INFO:tensorflow:Assets written to: .\T2\North\0.2\model\assets


0.22319456870016913
Threshold: 0
Intersections included: 13
Epoch 00022: early stopping
Epoch 00022: early stopping


INFO:tensorflow:Assets written to: .\T2\North\0\model\assets


INFO:tensorflow:Assets written to: .\T2\North\0\model\assets


0.3437427379056042


In [ ]:
#t1 south 0 is the same as 0.3, but add it so function lower down doesn't break bc data is missing
lstm['T1']['South'][0] = lstm['T1']['South'][0.3]

In [5]:
with open('LSTM_results_april.pickle', 'wb') as file:
    pickle.dump([lstm,data],file)  

In [6]:
lstm

{'T1': {'North': {1: {'intersections': ['K504'],
    'df':            Real  Predicted        SE
    0     -0.856211  -0.200575  0.429858
    1     -0.964397  -0.149855  0.663479
    2     -0.928335  -0.121102  0.651626
    3     -1.180771  -0.105568  1.156060
    4     -1.180771  -0.115106  1.135642
    ...         ...        ...       ...
    26740 -1.144709  -0.233276  0.830710
    26741 -1.180771  -0.224233  0.914964
    26742 -1.180771  -0.214655  0.933379
    26743 -1.180771  -0.224439  0.914571
    26744 -1.288957  -0.232424  1.116264
    
    [26745 rows x 3 columns],
    'MSE': 0.3427815773198963},
   0.8: {'MSE': 0.3427815773198963},
   0.7: {'MSE': 0.3427815773198963},
   0.6: {'MSE': 0.3427815773198963},
   0.5: {'intersections': ['K504', 'K559', 'K561'],
    'df':            Real  Predicted        SE
    0     -0.856211  -0.459764  0.157170
    1     -0.964397  -0.356525  0.369509
    2     -0.928335  -0.370402  0.311290
    3     -1.180771  -0.359620  0.674288
    4     -1

In [7]:
def plot_results(errors):
    df = pd.DataFrame()
    row = {}
    for trajectory in errors.keys():
        for direction in errors[trajectory].keys():
            row['model']="T{}_{}".format(trajectory,direction[0])
            for threshold in errors[trajectory][direction].keys():
                row["T{}".format(threshold)] = round(errors[trajectory][direction][threshold]['MSE'],3)
            df = df.append(row,ignore_index=True)
    return df
test = plot_results(lstm)

In [9]:
test.describe()

,T0,T0.2,T0.3,T0.4,T0.5,T0.6,T0.7,T0.8,T1
count,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
mean,0.280750,0.275500,0.255500,0.225750,0.279500,0.252750,0.252750,0.252750,0.252750
std,0.049561,0.039745,0.059383,0.062713,0.102468,0.138949,0.138949,0.138949,0.138949
min,0.233000,0.223000,0.189000,0.176000,0.149000,0.120000,0.120000,0.120000,0.120000
25%,0.246500,0.257500,0.214500,0.185750,0.245000,0.141750,0.141750,0.141750,0.141750
50%,0.273000,0.282000,0.259000,0.206000,0.285000,0.246000,0.246000,0.246000,0.246000
75%,0.307250,0.300000,0.300000,0.246000,0.319500,0.357000,0.357000,0.357000,0.357000
max,0.344000,0.315000,0.315000,0.315000,0.399000,0.399000,0.399000,0.399000,0.399000


In [10]:
test

,T0,T0.2,T0.3,T0.4,T0.5,T0.6,T0.7,T0.8,T1,model
0,0.233,0.269,0.189,0.189,0.293,0.343,0.343,0.343,0.343,TT1_N
1,0.295,0.295,0.295,0.176,0.399,0.399,0.399,0.399,0.399,TT1_S
2,0.251,0.315,0.315,0.315,0.149,0.149,0.149,0.149,0.149,TT2_S
3,0.344,0.223,0.223,0.223,0.277,0.120,0.120,0.120,0.120,TT2_N


In [20]:
test.append(test.describe().iloc[1])

,T0,T0.2,T0.3,T0.4,T0.5,T0.6,T0.7,T0.8,T1,model
0,0.23300,0.2690,0.1890,0.18900,0.2930,0.34300,0.34300,0.34300,0.34300,TT1_N
1,0.29500,0.2950,0.2950,0.17600,0.3990,0.39900,0.39900,0.39900,0.39900,TT1_S
2,0.25100,0.3150,0.3150,0.31500,0.1490,0.14900,0.14900,0.14900,0.14900,TT2_S
3,0.34400,0.2230,0.2230,0.22300,0.2770,0.12000,0.12000,0.12000,0.12000,TT2_N
mean,0.28075,0.2755,0.2555,0.22575,0.2795,0.25275,0.25275,0.25275,0.25275,NaN


In [12]:
data.keys()

dict_keys(['T1_North', 'T1_South', 'T2_South', 'T2_North'])

In [14]:
lstm.keys()

dict_keys(['T1', 'T2'])

In [18]:
lstm['T1']['North'][1]['df']

,Real,Predicted,SE
0,-0.856211,-0.200575,0.429858
1,-0.964397,-0.149855,0.663479
2,-0.928335,-0.121102,0.651626
3,-1.180771,-0.105568,1.156060
4,-1.180771,-0.115106,1.135642
...,...,...,...
26740,-1.144709,-0.233276,0.830710
26741,-1.180771,-0.224233,0.914964
26742,-1.180771,-0.214655,0.933379
26743,-1.180771,-0.224439,0.914571


In [42]:
a = data['T1_North'].iloc[-len(lstm['T1']['North'][1]['df'])].name

In [38]:
results['T1']results['T1']['North']['lof_df'].index>pd.Timestamp('2019-12-22 07:15:00')

array([False, False, False, ...,  True,  True,  True])

In [163]:
def outlier_mse_calc(lstm=lstm,data=data,results=results,outliers=5,target_intersections=target_intersections):
    """calculates MSE per model for top 'outliers' in test set"""
    mse_results=pd.DataFrame(columns=['Model','T 1.0','T 0.4','T 0.0'])
    thresholds = [1,0.4,0]
    for trajectory in ['T1','T2']:
        for direction in results[trajectory].keys():
            mses=[]
            for threshold in thresholds:
                #get preds, rawdata & lof dfs:
                preds = lstm[trajectory][direction][threshold]['df'].copy(deep=True)
                rawdata = data['{}_{}'.format(trajectory, direction)].iloc[-len(preds):].copy(deep=True)
                lof = results[trajectory][direction]['lof_df'].copy(deep=True)
                #take LOF df:
                lof = lof[lof.index> rawdata.iloc[0].name]
                
                #sort LOF df on outliers & take top 5 outliers for target intersection:
                lof.sort_values(by=target_intersections[trajectory][direction], ascending=False, inplace=True)
                outliers = lof.iloc[:5].index
                
                #insert dates on lstm preds df:
                preds['Time'] = rawdata.index
                outlier_df= pd.DataFrame()

                for outlier in outliers:
                    one_outlier = preds[(preds['Time']>=outlier) & (preds['Time']<(outlier+pd.Timedelta('1H')))]
                    outlier_df = pd.concat([outlier_df,one_outlier])
                mses.append(round(outlier_df['SE'].mean(),3))
            mse_results = mse_results.append({'Model':"{}_{}".format(trajectory,direction),
                                'T 1.0':mses[0],'T 0.4':mses[1],'T 0.0':mses[2]},ignore_index=True)
    return mse_results

In [164]:
mse_results = outlier_mse_calc()

In [165]:
mse_results

,Model,T 1.0,T 0.4,T 0.0
0,T1_North,0.776,0.371,0.532
1,T1_South,0.989,0.422,0.720
2,T2_South,0.195,0.704,0.585
3,T2_North,0.160,0.173,0.201


In [166]:
mse_results = mse_results.append({'Model':'Mean','T 1.0':round(mse_results.describe().iloc[1]['T 1.0'],3),
                                  "T 0.4":round(mse_results.describe().iloc[1]['T 0.4'],3),
                                  'T 0.0':round(mse_results.describe().iloc[1]['T 0.0'],3)},ignore_index=True)

In [167]:
mse_results

,Model,T 1.0,T 0.4,T 0.0
0,T1_North,0.776,0.371,0.532
1,T1_South,0.989,0.422,0.720
2,T2_South,0.195,0.704,0.585
3,T2_North,0.160,0.173,0.201
4,Mean,0.530,0.418,0.509


In [168]:
print(mse_results.to_latex())

\begin{tabular}{llrrr}
\toprule
{} &     Model &  T 1.0 &  T 0.4 &  T 0.0 \\
\midrule
0 &  T1\_North &  0.776 &  0.371 &  0.532 \\
1 &  T1\_South &  0.989 &  0.422 &  0.720 \\
2 &  T2\_South &  0.195 &  0.704 &  0.585 \\
3 &  T2\_North &  0.160 &  0.173 &  0.201 \\
4 &      Mean &  0.530 &  0.418 &  0.509 \\
\bottomrule
\end{tabular}



In [169]:
#nice. Should also change results for other stuff in the paper. 

In [170]:
#save all stuff extra:
all_stuff = [lstm,data,results]
with open('march30_save.pickle', 'wb') as f:
    pickle.dump(all_stuff,f)
    